<a href="https://colab.research.google.com/github/asadimtiazmalik/EmotionbasedMusicGeneration/blob/main/Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Cloning the emopia repository**

In [1]:
%%shell
git clone https://github.com/annahung31/EMOPIA.git

Cloning into 'EMOPIA'...
remote: Enumerating objects: 805, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 805 (delta 0), reused 0 (delta 0), pack-reused 804
Receiving objects: 100% (805/805), 33.35 MiB | 37.65 MiB/s, done.
Resolving deltas: 100% (401/401), done.


#**Installing dependencies**

In [2]:
%%writefile /content/EMOPIA/requirements.txt
scikit-learn==0.24.1
seaborn==0.11.1
numpy==1.19.5
miditoolkit==0.1.14
pandas==1.1.5
ipdb
tqdm
matplotlib
scipy

Overwriting /content/EMOPIA/requirements.txt


##**Running the `requirements.txt` file**

In [ ]:
%%shell
cd EMOPIA/
pip install -r requirements.txt

##**Installing `pytorch-fast-transformers`, `muspy` and `midiSynth` modules**

In [ ]:
%%shell
# pip install torch
pip install --user pytorch-fast-transformers -U

**Here we save all the models provided by the emopia authors for conditional music generation**

In [5]:
%%shell 
cd EMOPIA/
mkdir exp
gdown --id 1Q9vQYnNJ0hXBFwcxdWQgDNmzoW3MLl3h --output exp/baseline.zip
gdown --id 1ZULJgBRu2Wb3jxFmGfAHP1v_tjoryFM7 --output exp/no-pretrained_transformer.zip
gdown --id 19Seq18b2JNzOamEQMG1uarKjj27HJkHu --output exp/pretrained_transformer.zip 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Q9vQYnNJ0hXBFwcxdWQgDNmzoW3MLl3h
To: /content/EMOPIA/exp/baseline.zip
100% 429M/429M [00:05<00:00, 73.9MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ZULJgBRu2Wb3jxFmGfAHP1v_tjoryFM7
To: /content/EMOPIA/exp/no-pretrained_transformer.zip
100% 182M/182M [00:02<00:00, 73.5MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=Fu

In [ ]:
!pip install muspy

In [10]:
%cd EMOPIA/

/content/EMOPIA


In [11]:
%cd ..

/content


**Crating the data directory to store the dataset**

In [12]:
%%shell
cd EMOPIA/
mkdir data
cd data
mkdir emopia 

In [13]:
import muspy 
emopia = muspy.EMOPIADataset("/content/EMOPIA/data/emopia", download_and_extract=True)
emopia.convert()
music = emopia[0]
print(music.annotations[0].annotation)

62988951552it [00:07, 8311814417.59it/s] 


Successfully downloaded source : /content/EMOPIA/data/emopia/EMOPIA_2.2.zip .
Extracting archive : /content/EMOPIA/data/emopia/EMOPIA_2.2.zip ...
Successfully extracted archive : /content/EMOPIA/data/emopia .
Converting and saving the dataset...


100%|██████████| 1071/1071 [00:09<00:00, 107.62it/s]

Successfully saved 1071 out of 1071 files.
{'emo_class': '1', 'YouTube_ID': '0vLPYiPN7qY', 'seg_id': '0'}


In [ ]:
!pip install midiSynth

#**Generating music using the Compound Word transformer + LSTM Architecture**

**Music is generated based on the `emotion_tags` generated in the last notebook. These emotion tags essentially represent a quadrant as mentioned in the last notebook**

In [1]:
from midiSynth.synth import MidiSynth
midi_synth = MidiSynth()

In [2]:
%cd /content/EMOPIA/workspace/transformer

/content/EMOPIA/workspace/transformer


In [3]:
import os
import pickle
import torch
from utils import write_midi
from models import TransformerModel, network_paras

In [4]:
!gdown --id 17dKUf33ZsDbHC5Z6rkQclge3ppDTVCMP
!unzip co-representation.zip -d ../../dataset/
!rm co-representation.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=17dKUf33ZsDbHC5Z6rkQclge3ppDTVCMP
To: /content/EMOPIA/workspace/transformer/co-representation.zip
100% 16.6M/16.6M [00:00<00:00, 36.3MB/s]
Archive:  co-representation.zip
  inflating: ../../dataset/co-representation/ailabs_data.npz  
  inflating: ../../dataset/co-representation/ailabs_fn2idx_map.json  
  inflating: ../../dataset/co-representation/dictionary.pkl  
  inflating: ../../dataset/co-representation/emopia_data.npz  
  inflating: ../../dataset/co-representation/emopia_fn2idx_map.json  
  inflating: ../../dataset/co-representation/emopia_idx.npz  
  inflating: ../../dataset/co-representation/README.md  


In [5]:
path_dictionary = '../../dataset/co-representation/dictionary.pkl'
assert os.path.exists(path_dictionary)

In [6]:
dictionary = pickle.load(open(path_dictionary, 'rb'))
event2word, word2event = dictionary

In [7]:
# config
n_class = []   # num of classes for each token
for key in event2word.keys():
    n_class.append(len(dictionary[0][key]))
n_token = len(n_class)

In [8]:
%cd ..

/content/EMOPIA/workspace


In [9]:
%cd ..

/content/EMOPIA


In [10]:
!gdown --id 19Seq18b2JNzOamEQMG1uarKjj27HJkHu --output exp/pretrained_transformer.zip
!unzip exp/pretrained_transformer.zip -d exp/
!rm exp/pretrained_transformer.zip
os.listdir('exp/pretrained_transformer')

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19Seq18b2JNzOamEQMG1uarKjj27HJkHu
To: /content/EMOPIA/exp/pretrained_transformer.zip
100% 182M/182M [00:00<00:00, 196MB/s]
Archive:  exp/pretrained_transformer.zip
   creating: exp/pretrained_transformer/
  inflating: exp/pretrained_transformer/loss_25_params.pt  


['loss_25_params.pt']

In [11]:
%cd /content/EMOPIA/workspace/transformer

/content/EMOPIA/workspace/transformer


In [12]:
path_saved_ckpt = '/content/EMOPIA/exp/pretrained_transformer/loss_25_params.pt'
assert os.path.exists(path_saved_ckpt)

# **Model Definition and loading pretrained weights to the network**

In [13]:
# init model
net = TransformerModel(n_class, is_training=False)
net.cuda()
net.eval()

net.load_state_dict(torch.load(path_saved_ckpt))

>>>>>: [56, 135, 18, 4, 87, 18, 42, 5]
 [o] using RNN backend.


<All keys matched successfully>

In [14]:
emotion_tag = 2  # the target emotion class output from the images in the last phase. 
path_outfile = 'test' # output midi file name

##**Writing generated music to `.mid` file**

In [ ]:
res, _ = net.inference_from_scratch(dictionary, emotion_tag, n_token=8, display=False)
write_midi(res, path_outfile + '.mid', word2event)

------ initiate ------
0               | 0               | 0               | Emotion         | 0               | 0               | 0               | 1               | 
0               | 0               | Bar             | Metrical        | 0               | 0               | 0               | 0               | 
------ generate ------

--------[Done]--------
(238, 8)


##**Install libraries for converting `.mid` file to `.mp3` format**

In [ ]:
%%bash
sudo apt-get install timidity
sudo apt-get install ffmpeg

##**Converting and saving the resulting `.mp3` file**

In [ ]:
%%bash
timidity test.mid -Ow -o - | ffmpeg -i - -acodec libmp3lame -ab 64k output_song.mp3


Playing test.mid
MIDI file: test.mid
Format: 1  Tracks: 2  Divisions: 480
Track name: piano
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable

#**Playing the generated piece based on Valence-Arousal Scores from Phase 1**

In [ ]:
from IPython.display import Audio
Audio('/content/EMOPIA/workspace/transformer/output_song.mp3')